In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
data = pd.read_csv('C:/Users/makam/Desktop/Capstone Project/diabetic_data.csv')
data1 = pd.read_csv('C:/Users/makam/Desktop/Capstone Project/diabetic_data.csv')
drugs = ['metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol',
       'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']
drug_d = pd.DataFrame()
for x in drugs:
    del data1[x]
    drug_d[x]=data[x]
    mapping_dict={x:{'No':0,'Down':1,'Steady':2,'Up':3}}
    drug_d.replace(mapping_dict,inplace=True)
data1['max_glu_serum'].replace({'None':0,'Norm':1,'>200':2,'>300':3},inplace=True)
data1['A1Cresult'].replace({'None':0,'Norm':1,'>7':2,'>8':3},inplace=True)
data1['change'].replace({'No':0,'Ch':1},inplace=True)
data1['diabetesMed'].replace({'No':0,'Yes':1},inplace=True)
data1['readmitted'].replace({'NO':0,'<30':1,'>30':1},inplace=True)
drug_d['encounter_id']=data['encounter_id']
ids = data['encounter_id']
ids1 = pd.DataFrame(ids)
ids1['key'] = ids1.index
drug_dt=drug_d.T
insulin_data = drug_d['insulin']
drug_dt.drop(['insulin'],inplace=True)
cols = drug_dt.columns.values.tolist()
drug_dt.drop(['encounter_id'],inplace=True)
no_combo=[]
combo=[]
for x in cols:
    if(drug_dt[x].sum()==0):
        no_combo.append(x)
    else:
        combo.append(x)
combo1 = pd.DataFrame(combo)
combo1.rename(columns={0:'key'},inplace=True)
combo2 = pd.merge(combo1,ids1,on='key',how='inner')
del combo2['key']
no_combo1 = pd.DataFrame(no_combo)
no_combo1.rename(columns={0:'key'},inplace=True)
no_combo2 = pd.merge(no_combo1,ids1,on='key',how='inner')
del no_combo2['key']
ins_data=pd.DataFrame(insulin_data,columns=['insulin'])
ins_data['encounter_id']=data['encounter_id']
combo3 = pd.merge(combo2,ins_data,on='encounter_id',how='inner')
no_combo3 = pd.merge(no_combo2,ins_data,on='encounter_id',how='inner')
no_diabetes = no_combo3[no_combo3['insulin']==0]
type1 = no_combo3[no_combo3['insulin']!=0]
type2 = combo3[combo3['insulin']==0]
t1t2 = combo3[combo3['insulin']!=0]
diabetes = pd.merge(drug_d,data1,on='encounter_id',how='inner')
patient_data = pd.read_excel('C:/Users/makam/Desktop/Capstone Project/Paitent_details.xlsx')
patient_data['race'].replace({'Caucasian':0,'AfricanAmerican':1,'?':2,'Hispanic':3,'Other':4,'Asian':5},inplace=True)
patient_data['gender'].replace({'Female':0,'Male':1,'Unknown/Invalid':2},inplace=True)
patient_data['age'].replace({'[70-80)':0,'[60-70)':1,'[50-60)':2,'[80-90)':3,'[40-50)':4,'[30-40)':5,'[90-100)':6,'[20-30)':7,'[10-20)':8,'[0-10)':9},inplace=True)
del patient_data['weight']
final_diabetes = pd.merge(patient_data,diabetes,on='encounter_id',how='inner')
ndc = pd.merge(no_diabetes['encounter_id'],final_diabetes,on='encounter_id',how='inner')
t1 = pd.merge(type1['encounter_id'],final_diabetes,on='encounter_id',how='inner')
t2= pd.merge(type2['encounter_id'],final_diabetes,on='encounter_id',how='inner')
t12 = pd.merge(t1t2['encounter_id'],final_diabetes,on='encounter_id',how='inner')
ndc['treatment']=0
t1['treatment']=1
t2['treatment']=2
t12['treatment']=3
fd = t1

In [12]:
patients = fd['patient_nbr'].unique().tolist()
for i in patients:
    eids = [fd['patient_nbr']==i]['encounter_id'].tolist()
    for x in range(1,len(eids),1):
        sum1=[]
        for y in drugs:
            if((int(fd[fd['encounter_id']==eids[x-1]][y])!=0) or (int(fd[fd['encounter_id']==eids[x]][y])!=0)):
                if(int(fd[fd['encounter_id']==eids[x]][y])==3):
                    sum1.append(1)
                else:
                    sum1.append(-1)
                
                    
                
        if(sum(sum1)<=0):
            fd[fd['encounter_id']==eids[x]]['effect']=1
        else:
            fd[fd['encounter_id']==eids[x]]['effect']=0

TypeError: list indices must be integers or slices, not str

In [7]:
fd

,encounter_id,patient_nbr,race,gender,age,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,...,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,max_glu_serum,A1Cresult,change,diabetesMed,readmitted,treatment
0,149190,55629189,0,0,8,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
1,500364,82442376,0,1,5,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
2,35754,82637451,0,1,2,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
3,28236,89869032,1,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
4,42570,77586282,0,1,3,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
5,62256,49726791,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
6,73578,86328819,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
7,89682,107389323,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
8,150006,22864131,2,0,2,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
9,248916,115196778,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
